final project items
- descibe the data and a summary of the stats
- state the research question, and what exactly we're trying to express with the data
- what is the method? why this visual? why is this one chosen?
- what's next? what conclusion can be drawn? what comes next for analyzing the data?

selenium documentation (for reader reference): 
* https://selenium-python.readthedocs.io/
* https://www.selenium.dev/documentation/

In [355]:
#importing our libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import csv

In [356]:
#booting up selenium to automate the scraping of reddit

#defining headless mode (if we want to run this operation without gui)
options = webdriver.FirefoxOptions()
options.add_argument("-headless")
browser = webdriver.Firefox(options=options)

#defining non-headless mode
#browser= webdriver.Firefox()

In [357]:
#helper function to help us get the number of instances of words in each post
def countwords(textblock, keyword):
    textblock= textblock.lower()
    
    return textblock.count(keyword)

In [358]:
#helper function to open a link to a page

def openlink(url):
    browser.get(url) #open the url in browser (we will be using firefox)

In [359]:
#helper function to get title and body text of post
#also get info like time, link, etc which may be helpful

def getpostinfo(url):
    #opening the link
    openlink(url)
    time.sleep(3) #tell code to pause for a while so the page can load
    data= []#defining an empty list for post data we want to return
    
    #select all the posts on the page and get its contents in the form of a list of elements
    posts= browser.find_elements(By.CSS_SELECTOR, "div[class='rounded-3xl bg-surface-100-800-token max-w-5xl w-full p-4 variant-ghost-surface my-3']")

    #scraping results in a daily query
    for post in posts:
        print("iterating posts")
        postinfo= {} #empty dict for information on every post
        title= post.find_element(By.TAG_NAME, 'h1').text #get title
        #count up keywords instances in title
        unalivecount= countwords(title, 'unalive')
        killcount= countwords(title, 'kill')
        
        #selecting the body text of the post
        fullbody= post.find_element(By.CSS_SELECTOR, "div[class='mt-2 overflow-hidden']")
        lines= fullbody.find_elements(By.CSS_SELECTOR, "p")
        bodytext=""
        for line in lines:
            bodytext+= ("\n"+line.text)
        
        #counting instances of the keywords in post body and adding it to total
        unalivecount+= countwords(bodytext, 'unalive')
        killcount+= countwords(bodytext, 'kill')
            
        footnote= post.find_elements(By.CSS_SELECTOR, "p[class='text-xs font-semibold'")
        
        postinfo['time']= footnote[3].text #TODO: WRITE HELPER FUNCTION TO CONVERT DATE TO USEFUL FORMAT
        postinfo['unalivecount']= unalivecount
        postinfo['killcount']= killcount
        postinfo['title']= title
        postinfo['text']= bodytext
        #postinfo['link']= footnote[4].text
        
        data.append(postinfo)
    
    return data

In [360]:
#reddit caps search results at 100, so instead we will separate our query by day and scrape the results

#defining the day we want to start querying. we will query every day from 1/1/2015 to 1/31/2025
#january 1 2015 (1/1/2015) at 11:59 pm
before= 1419984000
#december 31 2014 (12/31/2014) at 12:00 am
after= 1419897600
#each day is incremented by 86400 seconds

#TO DO: USE PULL PUSH IO INSTEAD OF REDDIT
#SCRAPE FOR EACH DAY AND LOG DATE AND CONTENT OF THE POST

#list of scraped post data, which will be populated with the text that gets scraped
postdata= {}

url= "https://search-new.pullpush.io/?subreddit=offmychest&type=submission&q=kill&sort_type=created_utc&sort=desc&before=1419897600&after=1419811200"
scrapedata= getpostinfo(url)
postdata['datetime']= {'numposts' : len(scrapedata), 'data' : scrapedata}
    
#print(getpostinfo(url, keyword))

'''
{'unalive' : [], 'kill' : [], 'suicide' : [], 'murder' : []}
while before<1738389600:
    url= f"https://search-new.pullpush.io/?subreddit=offmychest&type=submission&q=unalive&sort_type=created_utc&sort=asc&before={before}&after={after}"
    scrapedata= getpostinfo(url)
    postdata['unalive'].append({'time' : before, 'numposts' : len(scrapedata), 'data' : scrapedata})
    
    url= f"https://search-new.pullpush.io/?subreddit=offmychest&type=submission&q=kill&sort_type=created_utc&sort=asc&before={before}&after={after}"
    scrapedata= getpostinfo(url)
    postdata['kill'].append('time' : before, 'numposts' : len(scrapedata), 'data' : scrapedata})
    
    url= f"https://search-new.pullpush.io/?subreddit=offmychest&type=submission&q=suicide&sort_type=created_utc&sort=asc&before={before}&after={after}"
    scrapedata= getpostinfo(url)
    postdata['suicide'].append('time' : before, 'numposts' : len(scrapedata), 'data' : scrapedata})
    
    url= f"https://search-new.pullpush.io/?subreddit=offmychest&type=submission&q=murder&sort_type=created_utc&sort=asc&before={before}&after={after}"
    scrapedata= getpostinfo(url)
    postdata['murder'].append('time' : before, 'numposts' : len(scrapedata), 'data' : scrapedata})
    
    after=before
    before=before+86400

'''

#after collecting data, export it into csv format
with open('data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(postdata.keys())
    rows = zip(postdata.values())
    writer.writerows(rows)
    
browser.quit()


iterating posts
iterating posts
iterating posts
iterating posts
iterating posts
iterating posts
iterating posts
iterating posts
iterating posts
iterating posts
iterating posts
iterating posts
iterating posts
iterating posts
iterating posts
iterating posts
iterating posts
